In [1]:
import nlp516

In [2]:
dataset = nlp516.data.PublicTrial()

In [3]:
dataset.es

,id,text,HS,TR,AG
0,101,@ian_delaCalva @IrantzuVarela @pikaramagazine ...,0,0,0
1,102,NINGUNA MUJER ES 'PUTA' ❗❗❗ https://t.co/cV0CQ...,0,0,0
2,103,"Editar, además de complicado, es lo que hace d...",0,0,0
3,104,Bien joder una puta alegría!! Te lo mereces pe...,0,0,0
4,105,Todo política... no se levanta la sesión y est...,0,0,0
5,106,Casuso callate de una puta vez https://t.co/tW...,0,0,0
6,107,"@BedMM @NayBetania Tú eres la culpable, perra >:v",1,1,1
7,108,Yo no puedo creer que las propias mujeres natu...,0,0,0
8,109,@DalasReview @jk_rowling Hijo de puta muerete ...,0,0,0
9,110,@Wachit0 @100Akshat @_Midoriyaa_ @NeohSJ La id...,0,0,0
